# <font size=35 color=lightgreen>** Sentiment API **<font>🥲
---

### <font size=12 color=lightgreen>Configuración Inicial (Librerías)</font>

#### 1. Procesamiento y Manipulación de Datos
* **`pandas`**
    * Nos ayuda con la manipulación y análisis de datos estructurados.
    * Carga el dataset (CSV), gestiona el DataFrame y permite filtrar o limpiar registros.
* **`numpy`**
    * Realiza las operaciones matemáticas y manejo de arrays eficientes.
    * Soporte numérico fundamental para las transformaciones vectoriales de los textos.

#### 2. Visualización y Análisis Exploratorio

* **`matplotlib.pyplot`**
    * Generación de gráficos estáticos.
    * Visualización básica de la distribución de clases (Positivo vs. Negativo).
* **`seaborn`**
    * Visualización de datos estadísticos avanzada.
    * Generación de matrices de confusión y gráficos de distribución estéticos para la presentación.

#### 3. Procesamiento de Lenguaje Natural (NLP) y Limpieza

* **`re`** (Regular Expressions)
    * Manejo de expresiones regulares.
    * Eliminación de ruido en el texto: URLs, menciones (@usuario), hashtags (#) y caracteres especiales no alfanuméricos.
* **`string`**
    * Constantes de cadenas comunes.
    * Provee listas estándar de signos de puntuación para su eliminación eficiente.

#### 4. Modelado y Machine Learning (Core)

* **`scikit-learn`**
    * Biblioteca principal de Machine Learning.
    * **`TfidfVectorizer`**: Transforma el texto limpio en vectores numéricos.
    * **`LogisticRegression`**: Algoritmo de clasificación supervisada.
    * **`metrics`**: Cálculo de precisión, recall y F1-score.
    * **`Pipeline`**: Encapsulamiento de los pasos de transformación y predicción.

#### 5. Persistencia e Integración
Herramientas para conectar el modelo con el Backend.

* **`joblib`**
    * Serialización eficiente de objetos Python.
    * Exportar (`dump`) el pipeline entrenado a un archivo `.joblib` y cargarlo (`load`) en la API para realizar predicciones.
* **`fastapi` & `uvicorn`**
    * Framework web moderno de alto rendimiento.
    * Exponer el modelo entrenado como un microservicio REST (endpoint `/predict`) para ser consumido por el Backend en Java.




---



### <font size=16  color=lightgreen> Importando librerías <font>



In [ ]:

import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import re
import string
import chardet
import sklearn
import fastapi
import joblib
import nltk
import unicodedata
import urllib.request
import urllib.error
from io import StringIO
import urllib.response
import os
from pathlib import Path
from datetime import datetime
import warnings






### <font size = 8 color="lightgreen">Importación de los datasets<font>

#### **Función importación dataset**

In [216]:
warnings.filterwarnings("ignore")
def importar_dataset(url, separator=';'):
    """
    Importa dataset desde URL detectando encoding automáticamente.
    """
    try:
        # 1. Descargar contenido una sola vez
        with urllib.request.urlopen(url) as response:
            content = response.read()

        # 2. Detectar encoding
        result = chardet.detect(content)
        encoding = result['encoding']
        print(f"🔍 Encoding detectado: {encoding} (confianza: {result['confidence']:.2%})")

        # 3. Decodificar y cargar en DataFrame
        decoded_content = content.decode(encoding, errors='replace')
        data = pd.read_csv(StringIO(decoded_content), sep=separator)

        print("✅ Archivo cargado correctamente")
        print(f"📊 Tamaño del dataset: {data.shape}")
        print("\n🔍 Muestra aleatoria (3 registros):")
        print(data.sample(3))

        return data

    except urllib.error.URLError as e:
        print(f"❌ Error de URL: {e}")
        return None
    except pd.errors.ParserError as e:
        print(f"❌ Error al parsear CSV: {e}")
        return None
    except Exception as e:
        print(f"❌ Error inesperado: {type(e).__name__}: {e}")
        return None

#### **Dataset1: sentimentdataset_es.csv**

In [217]:
df1_raw = importar_dataset("https://github.com/ml-punto-tech/sentiment-api/raw/refs/heads/dev/data-science/datasets/datasets-origin/sentimentdataset_es.csv")


🔍 Encoding detectado: Windows-1252 (confianza: 72.97%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (731, 15)

🔍 Muestra aleatoria (3 registros):
     Unnamed: 0.1  Unnamed: 0  \
523           525         529   
164           165         167   
288           289         293   

                                                  Text  Sentiment  \
523  En el público de una actuación de Jay-Z, la le...    Orgullo   
164  Abrumado por el dolor, extrañando profundament...      Dolor   
288  Susurros Esperanzadores del viento, que llevan...  Esperanza   

            Timestamp                User   Platform  \
523  10-09-2017 20:15  JayZEmpireListener    Twitter   
164  05-07-2022 15:45       MourningHeart  Instagram   
288   06-05-2023 7:20       WindWhisperer    Twitter   

                             Hashtags  Retweets  Likes Country  Year  Month  \
523              #Orgullo #JayZAnthem        18     35   EE.UU  2017      9   
164                   #Duelo #Pérdida         8     

#### **Dataset2: sentiment_analysis_dataset.csv**

In [218]:
df2_raw = importar_dataset("https://raw.githubusercontent.com/ml-punto-tech/sentiment-api/refs/heads/feature/data-science-marely/data-science/datasets/datasets-origin/sentiment_analysis_dataset.csv")

🔍 Encoding detectado: Windows-1252 (confianza: 73.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (2540, 3)

🔍 Muestra aleatoria (3 registros):
                                                  texto  label sentimiento
1104  Todo el dia intentando compras las entradas pa...      0    negativo
1829         Hoy he decidido, que te tengo que olvidar.      1     neutral
1949  Hoy me dijeron que yo era una niña EXITOSA, y ...      1     neutral


In [219]:
df3_raw = importar_dataset("https://github.com/eduardotec05/datasets/raw/refs/heads/main/twitter_training_esp_convertido%20(2).csv", separator=',')

🔍 Encoding detectado: UTF-8-SIG (confianza: 100.00%)
✅ Archivo cargado correctamente
📊 Tamaño del dataset: (74682, 4)

🔍 Muestra aleatoria (3 registros):
          id              plataforma sentimiento  \
52467  10613  RedDeadRedemption(RDR)     Neutral   
14040   2808                   Dota2     Neutral   
22172   4194                   CS-GO     Neutral   

                                                   texto  
52467  Red Dead Redemption. Nuevo en store.playstatio...  
14040  La temporada 3 de OGA Dota PIT: Europa/CIS par...  
22172  CS: GO: Agrega nuevo banco en Mirage. Smileybs...  


#### **Dataset3: Twitter_training.csv**

In [220]:
df3_raw.head()

,id,plataforma,sentimiento,texto
0,2401,Borderlands,Positivo,Estoy llegando a Borderlands y los asesinaré a...
1,2401,Borderlands,Positivo,"Voy a llegar a las fronteras y os mataré a todos,"
2,2401,Borderlands,Positivo,Voy a llegar a Borderlands y los mataré a todos.
3,2401,Borderlands,Positivo,Voy a llegar a Borderlands y los asesinaré a t...
4,2401,Borderlands,Positivo,Me estoy metiendo en Borderlands 2 y os voy a ...


In [221]:
def verificar_calidad_importacion(df, nombre_dataset):
    """
    Verifica que no se haya perdido información durante la importación.
    """
    print(f"\n🔍 VERIFICACIÓN DE CALIDAD: {nombre_dataset}")
    print("=" * 60)

    if df is None:
        print("❌ Dataset es None")
        return False

    # 1. Información básica
    print(f"📊 Forma: {df.shape}")
    print(f"📝 Columnas: {list(df.columns)}")

    # 2. Buscar columnas de texto
    columnas_texto = [col for col in df.columns if df[col].dtype == 'object']
    print(f"🔤 Columnas de texto: {columnas_texto}")

    if not columnas_texto:
        print("⚠️  No se encontraron columnas de texto")
        return True

    # 3. Analizar una columna de texto (usar la primera)
    col_texto = columnas_texto[0]
    print(f"\n📝 Analizando columna: '{col_texto}'")

    # Muestra de textos
    textos = df[col_texto].dropna().head(5).tolist()

    problemas = []

    for i, texto in enumerate(textos):
        if isinstance(texto, str):
            # Buscar caracteres de reemplazo (�) que indican problemas
            caracteres_problema = texto.count('�')
            if caracteres_problema > 0:
                problemas.append(f"Texto {i+1} tiene {caracteres_problema} caracteres de reemplazo (�)")

            # Buscar emojis
            emojis = [c for c in texto if unicodedata.category(c)[0] in ['S', 'So']]
            if emojis:
                print(f"  Texto {i+1}: ✅ Tiene {len(emojis)} emoji(s): {''.join(emojis[:3])}")
            else:
                print(f"  Texto {i+1}: 📄 Sin emojis")

            # Mostrar fragmento
            preview = texto[:80] + "..." if len(texto) > 80 else texto
            print(f"     '{preview}'")

    # 4. Resumen
    if problemas:
        print(f"\n⚠️  PROBLEMAS ENCONTRADOS:")
        for problema in problemas:
            print(f"   • {problema}")
        return False
    else:
        print(f"\n✅ CALIDAD OK: No se detectaron caracteres perdidos")
        return True

verificar_calidad_importacion(df1_raw, "Dataset 1")
verificar_calidad_importacion(df2_raw, "Dataset 2")
verificar_calidad_importacion(df3_raw, "Dataset 3")


🔍 VERIFICACIÓN DE CALIDAD: Dataset 1
📊 Forma: (731, 15)
📝 Columnas: ['Unnamed: 0.1', 'Unnamed: 0', 'Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Retweets', 'Likes', 'Country', 'Year', 'Month', 'Day', 'Hour']
🔤 Columnas de texto: ['Text', 'Sentiment', 'Timestamp', 'User', 'Platform', 'Hashtags', 'Country']

📝 Analizando columna: 'Text'
  Texto 1: 📄 Sin emojis
     '¡Disfrutando de un hermoso día en el parque!'
  Texto 2: 📄 Sin emojis
     'Esta mañana el tráfico era terrible.'
  Texto 3: 📄 Sin emojis
     '¡Acabo de terminar un entrenamiento increíble!??'
  Texto 4: 📄 Sin emojis
     '¡Emocionado por la escapada de fin de semana que viene!'
  Texto 5: 📄 Sin emojis
     'Probando una nueva receta para cenar esta noche.'

✅ CALIDAD OK: No se detectaron caracteres perdidos

🔍 VERIFICACIÓN DE CALIDAD: Dataset 2
📊 Forma: (2540, 3)
📝 Columnas: ['texto', 'label', 'sentimiento']
🔤 Columnas de texto: ['texto', 'sentimiento']

📝 Analizando columna: 'texto'
  Texto 1: 📄 Sin e

True

<font color='lightgreen' size=12>Filtrar datasets</font>

In [222]:
# Función filtrar dataset
def filtrar_dataset(data):
    data_filtro = data[['texto', 'sentimiento']]
    data_filtro = data_filtro[data_filtro['texto'].str.strip() != ""]
    print(data_filtro.sample(5))
    return data_filtro

# Reemplazar nombre columnas Text por texto, Sentiment por sentimiento
df1_raw.rename({'Text':'texto', 'Sentiment':'sentimiento'}, axis=1, inplace=True)
df1_filtrado = filtrar_dataset(df1_raw)
df2_filtrado = filtrar_dataset(df2_raw)
df3_filtrado = filtrar_dataset(df3_raw)

                                                 texto  sentimiento
599  Pasé la tarde en un museo, fingiendo ser culto...   Curiosidad
682  Reflexionando sobre los desafíos del año escol...     Tristeza
675  Elaboración de intrincadas pulseras de la amis...      Alegría
450  Asfixiándose en el silencio de la soledad, don...  Aislamiento
496  Viajando a través de la serenidad de Santorini...    Serenidad
                                                  texto sentimiento
1864  "No son momentos de confusión, son momentos de...     neutral
1531  Hay algo de mi por lo que me siento muy orgull...    positivo
1957  Gracias Por Su Apoyo Gente Gracias A Ustedes Y...     neutral
819   Mi columna de mañana en La República se llama ...    negativo
363   #gelp Pero claro. Ahora hay q tomar sopa.  A p...    negativo
                                                   texto  sentimiento
70058      @fox_tica Dutch nos advirtió que tenía razón!      Neutral
14054  Geek Fam gana la ONE Esports Dota 2 S

### <font size= 12 color="lightgreen" >Explorando los datasets<font>

In [223]:
# Crear función para explorar datasets
def explorar_dataset(data):
    print('Filas: ' + str(data.shape[0]))
    print('Columnas: ' + str(data.shape[1]))
    print('\nColumnas: \n' + str(data.columns.tolist()))
    print('\nTipo de datos: \n' + str(data.dtypes))
    print('\nValores nulos: \n' + str(data.isnull().sum()))
    print('\nMuestra aleatoria (5 registros): \n' + str(data.sample(5)))

#### **Explorando Data1**

In [224]:
explorar_dataset(df1_filtrado)

Filas: 731
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
666  Envié accidentalmente un mensaje de texto dest...   Verguenza
207  Flotando durante el día con un aire de indifer...     Neutral
519  En un concierto de Justin Bieber, los contagio...  Entusiasmo
665  Bailé bajo la lluvia para celebrar el final de...     Alegría
263  Con el corazón entusiasmado, corriendo por cam...       Ánimo


#### **Explorando data2**

In [225]:
explorar_dataset(df1_filtrado)

Filas: 731
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          0
sentimiento    0
dtype: int64

Muestra aleatoria (5 registros): 
                                                 texto sentimiento
656  Llevar bocadillos a clase como un profesional....    Positivo
384  Susurros de inspiración provenientes del susur...     Neutral
229  La melancolía pinta el mundo con tonos de nost...     Neutral
374  Persiguiendo sueños como una cometa volando al...    Positivo
556  A pesar del meticuloso entrenamiento, el nadad...    Negativo


In [226]:
explorar_dataset(df3_filtrado)

Filas: 74682
Columnas: 2

Columnas: 
['texto', 'sentimiento']

Tipo de datos: 
texto          object
sentimiento    object
dtype: object

Valores nulos: 
texto          41
sentimiento     0
dtype: int64

Muestra aleatoria (5 registros): 
                                                   texto  sentimiento
34187     No sé si deberían sentirse ofendidos por esto.     Negativo
36383  Game Pass está muerto, la mejor película que M...     Positivo
955    Â¿CÃ“MO HA SIDO TODO EL AÃ‘O? El aÃ±o pasado t...     Positivo
7399   Podría volver a jugar a los clásicos de WoW. L...     Negativo
42416  Las chicas que tocan en el pub y encienden el ...  Irrelevante


### <font size=12 color=lightgreen>Limpiar textos</font>

In [ ]:
# FUNCION SEGMENTADA DE LIMPIEZA BASICA PRUEBA.
texto = unicodedata.normalize('NFD', texto)

# Preservar ñ/Ñ
texto = texto.replace('n\\u0303', '@@@N_TILDE@@@')
texto = texto.replace('ñ', '@@@N_TILDE@@@')
texto = texto.replace('N\\u0303', '@@@N_TILDE_MAYUS@@@')
texto = texto.replace('Ñ', '@@@N_TILDE_MAYUS@@@')

# Eliminar tildes
texto = ''.join(char for char in texto if not unicodedata.combining(char))

# Restaurar ñ/Ñ
texto = texto.replace('@@@N_TILDE@@@', 'ñ')
texto = texto.replace('@@@N_TILDE_MAYUS@@@', 'Ñ')

# Normalizar espacios básico
texto = ' '.join(texto.split())
texto = texto.strip()

#### **Función para limpieza de textos**

In [227]:
def limpiar_texto_sentimientos(texto):
    """
    Normaliza texto español preservando ñ y eliminando tildes.
    NO convierte a minúsculas para preservar intensidad emocional.
    """
    # Verifica si la entrada no es una cadena. Si no lo es, devuelve una cadena vacía.
    if not isinstance(texto, str):
        return ""

    # 1. Normaliza el texto para separar los caracteres base de sus diacríticos (ej., tildes).
    texto = unicodedata.normalize('NFD', texto)

    # 2. Reemplaza temporalmente las 'ñ' y 'Ñ' con marcadores especiales para preservarlas
    # durante la eliminación de diacríticos.
    texto = texto.replace('n\u0303', '@@@N_TILDE@@@')
    texto = texto.replace('ñ', '@@@N_TILDE@@@')
    texto = texto.replace('N\u0303', '@@@N_TILDE_MAYUS@@@')
    texto = texto.replace('Ñ', '@@@N_TILDE_MAYUS@@@')

    # 3. Elimina los caracteres diacríticos (como las tildes) del texto.
    texto = ''.join(
        char for char in texto
        if not unicodedata.combining(char)
    )

    # Restaura las 'ñ' y 'Ñ' utilizando los marcadores temporales.
    texto = texto.replace('@@@N_TILDE@@@', 'ñ')
    texto = texto.replace('@@@N_TILDE_MAYUS@@@', 'Ñ')

    # Variable para almacenar el resultado de la limpieza.
    resultado = texto
    chars = []

    # Itera sobre cada caracter en el resultado y añade solo los caracteres imprimibles a una lista.
    # Los caracteres no imprimibles (como los de control) son reemplazados por un espacio.
    for char in resultado:
        if char.isprintable():
            chars.append(char)
        else:
            chars.append(' ')
    resultado = ''.join(chars)

    # Elimina URLs que terminan en "..." (posibles URLs rotas).
    resultado = re.sub(r'https?://[^\s]*\.\.\.', '[URL_ROTA]', resultado)
    resultado = re.sub(r'www\.[^\s]*\\.\\.\\.', '[URL_ROTA]', resultado)

    # Normaliza los espacios múltiples a uno solo y elimina espacios al inicio y final.
    resultado = ' '.join(resultado.split())
    resultado = resultado.strip()

    # Mostrar resultados estadísticos de la limpieza.




    # Devuelve el texto preprocesado.
    return resultado


#### **Análisis proceso de limpieza de textos**

In [228]:
def analizar_limpieza_sentimientos(df_antes, df_despues, nombre):
    """
    Análisis específico para tu función limpiar_texto_para_sentimientos
    """
    print(f"\n🔍 ANÁLISIS ESPECÍFICO: {nombre}")
    print("="*60)

    # 1. Cambios en caracteres específicos del español
    cambios_especificos = {
        'tildes_eliminadas': 0,
        'ñ_preservadas': 0,
        'urls_eliminadas': 0,
        'mayusculas_preservadas': 0
    }

    # Muestra de 50 textos para análisis detallado
    muestra = min(50, len(df_antes))

    for i in range(muestra):
        if i < len(df_despues):
            texto_antes = str(df_antes.iloc[i]['texto'])
            texto_despues = str(df_despues.iloc[i]['texto'])

            # Contar ñ preservadas
            if 'ñ' in texto_antes.lower() and 'ñ' in texto_despues.lower():
                cambios_especificos['ñ_preservadas'] += 1

            # Contar URLs eliminadas
            import re
            urls_antes = len(re.findall(r'https?://\S+', texto_antes))
            urls_despues = len(re.findall(r'https?://\S+', texto_despues))
            if urls_antes > urls_despues:
                cambios_especificos['urls_eliminadas'] += (urls_antes - urls_despues)

            # Verificar mayúsculas preservadas
            mayus_antes = sum(1 for c in texto_antes if c.isupper())
            mayus_despues = sum(1 for c in texto_despues if c.isupper())
            if mayus_antes > 0 and mayus_despues > 0:
                cambios_especificos['mayusculas_preservadas'] += 1

    print("📊 Cambios específicos de tu limpiador:")
    for cambio, cantidad in cambios_especificos.items():
        print(f"   • {cambio.replace('_', ' ').title()}: {cantidad} de {muestra} textos")



    print("="*60)

In [229]:
# Lista de dataframes para procesar
dataframes = [
    (df1_filtrado, "Dataset 1"),
    (df2_filtrado, "Dataset 2"),
    (df3_filtrado, "Dataset 3")
]

resultados = {}

for df, nombre in dataframes:
    # Aplicar limpieza
    df['Texto_Limpio'] = df['texto'].apply(limpiar_texto_sentimientos)

    # Guardar copia limpia
    resultados[nombre] = df.copy()

    # Mostrar info
    print(f"\n📁 {nombre}")
    print(f"   Registros: {len(df):,}")
    print(f"   Muestra (3 textos):")
    print(df[['texto', 'Texto_Limpio']].sample(3))

# Asignar a variables originales
df1_clean = resultados["Dataset 1"]
df2_clean = resultados["Dataset 2"]
df3_clean = resultados["Dataset 3"]

analizar_limpieza_sentimientos(df1_filtrado, df1_clean, "Dataset 1")
analizar_limpieza_sentimientos(df2_filtrado, df2_clean, "Dataset 2")
analizar_limpieza_sentimientos(df3_filtrado, df3_clean, "Dataset 3")


📁 Dataset 1
   Registros: 731
   Muestra (3 textos):
                                                 texto  \
568  Al capturar la esencia de un mercado bullicios...   
274  Susurros Esperanzaes del viento, que llevan la...   
487  En la celebración del éxito, los fuegos artifi...   

                                          Texto_Limpio  
568  Al capturar la esencia de un mercado bullicios...  
274  Susurros Esperanzaes del viento, que llevan la...  
487  En la celebracion del exito, los fuegos artifi...  

📁 Dataset 2
   Registros: 2,540
   Muestra (3 textos):
                                                  texto  \
2080  He concluido la experiencia de 'La sociedad de...   
1554  El soltar con gratitud te abre a nuevas posibi...   
2126  saber que con quien estuve románticamente me f...   

                                           Texto_Limpio  
2080  He concluido la experiencia de 'La sociedad de...  
1554  El soltar con gratitud te abre a nuevas posibi...  
2126  saber que co

In [230]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
174,El arrepentimiento por las oportunidades perdi...,Arrepentimiento,El arrepentimiento por las oportunidades perdi...
395,Deslumbrado por la elegancia de los deslumbran...,Deslumbrar,Deslumbrado por la elegancia de los deslumbran...
535,Al descubrir una joya escondida en el mundo de...,Positivo,Al descubrir una joya escondida en el mundo de...


In [231]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
1127,Quiero renunciar solo para tener mi liquidació...,negativo,Quiero renunciar solo para tener mi liquidacio...
1254,Deje para última hora un regalo que tenía en m...,positivo,Deje para ultima hora un regalo que tenia en m...
946,@2011rafaelhomez URGENTE detenido capitán reti...,negativo,@2011rafaelhomez URGENTE detenido capitan reti...


In [232]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
182,"Por alguna razón, mi computadora portátil pued...",Neutral,"Por alguna razon, mi computadora portatil pued..."
4797,buff.ly/33Q7U7s......,Neutral,buff.ly/33Q7U7s......
26631,Estoy muy cansado.),Positivo,Estoy muy cansado.)


### <font size=12 color=lightgreen>Categorizar de sentimientos </font>

#### **Limpieza de Sentimientos**

In [233]:
def limpiar_sentimiento_simple(sentimiento):
    """Convierte a minúsculas y quita espacios extras."""
    return " ".join(str(sentimiento).lower().strip().split())


def limpiar_y_unificar_sentimientos(lista_sentimientos, nombre_lista="lista"):
    """
    Recibe una lista (o iterable) de sentimientos y devuelve
    una lista ordenada de sentimientos únicos ya limpios.
    Además imprime un pequeño resumen.
    """
    # Sentimientos originales
    lista_sentimientos = list(lista_sentimientos)
    
    # Limpiar cada elemento
    limpios = (limpiar_sentimiento_simple(s) for s in lista_sentimientos)
    
    # Eliminar duplicados y ordenar
    sentimientos_limpios = sorted(set(limpios))

    # Prints usando el nombre de la lista
    print(f"📊 Lista '{nombre_lista}' (sin limpiar): {len(lista_sentimientos)}")
    print(f"Muestra (primeros 10): {lista_sentimientos[:10]}")
    print(f"📊 Lista '{nombre_lista}' (limpios): {len(sentimientos_limpios)}")
    print(f"Muestra (primeros 10): {sentimientos_limpios[:10]}")

    return sentimientos_limpios

# 1. Obtener sentimientos únicos de ambos datasets
sentimientos_unicos = sorted(
    list(df1_clean['sentimiento'].unique())
    + list(df2_clean['sentimiento'].unique())
    + list(df3_clean['sentimiento'].unique())
)

# 2. Aplicar la función genérica
sentimientos_unicos_limpios = limpiar_y_unificar_sentimientos(
    sentimientos_unicos,
    nombre_lista="sentimientos_unicos"
)



📊 Lista 'sentimientos_unicos' (sin limpiar): 111
Muestra (primeros 10): ['Abrumado', 'Aburrimiento', 'Aceptación', 'Admiración', 'Adoración', 'Agradecido', 'Aislamiento', 'Alegría', 'Amabilidad', 'Amargura']
📊 Lista 'sentimientos_unicos' (limpios): 105
Muestra (primeros 10): ['abrumado', 'aburrimiento', 'aceptación', 'admiración', 'adoración', 'agradecido', 'aislamiento', 'alegría', 'amabilidad', 'amargura']


#### **Categorías Sentimientos**

In [234]:
# 1. Definimos las listas de sentimientos según su categoría
print(f"Total de sentimientos únicos: {len(sentimientos_unicos_limpios)}")
print(sentimientos_unicos_limpios)


Total de sentimientos únicos: 105
['abrumado', 'aburrimiento', 'aceptación', 'admiración', 'adoración', 'agradecido', 'aislamiento', 'alegría', 'amabilidad', 'amargura', 'ambivalencia', 'amistad', 'amor', 'angustia', 'anhelo', 'ansiedad', 'anticipación', 'apreciación', 'aprensivo', 'armonía', 'arrepentimiento', 'asco', 'asombro', 'cautivación', 'celebración', 'colorido', 'confiado', 'confianza', 'contentamiento', 'creatividad', 'cumplimiento', 'curiosidad', 'decepción', 'desamor', 'descubrimiento', 'desesperación', 'deslumbrar', 'despectivo', 'determinación', 'devastado', 'disfrute', 'diversión', 'dolor', 'elegancia', 'emoción', 'empoderamiento', 'empático', 'encantamiento', 'energía', 'enojo', 'entumecimiento', 'entusiasmo', 'envidia', 'envidioso', 'esperanza', 'euforia', 'excitación', 'felicidad', 'frustración', 'frustrado', 'grandeza', 'gratitud', 'inspiración', 'inspirado', 'intimidación', 'irrelevante', 'juguetón', 'logro', 'lástima', 'malo', 'maravilla', 'melancolía', 'melódico',

In [235]:
# ==========================================================
# 🎯 CARGAR SENTIMIENTOS 
# ==========================================================

import json

# 1. Nombre del archivo (debe estar en la misma carpeta)

archivo = 'C:/Users/marely/OneDrive/Documentos/Oracle_ONE/Hackaton/SentimentAPI-Project/sentiment-api/data-science/sources/diccionarios/sentimientos_mapeo.json'
print(archivo)


# 2. Intentar cargar
try:
    with open(archivo, 'r', encoding='utf-8') as f:
        datos = json.load(f)
    
    print(f"✅ JSON cargado: {len(datos)} sentimientos")
    
    # 3. Crear listas directamente
    positivos = [k for k, v in datos.items() if v == 'positivo']
    negativos = [k for k, v in datos.items() if v == 'negativo']
    neutros = [k for k, v in datos.items() if v == 'neutral']
    
    print(f"📊 Positivos: {len(positivos)}, Negativos: {len(negativos)}, Neutros: {len(neutros)}")
    
except:
    print(f"❌ No se pudo cargar '{archivo}'")
    print("💡 Creando listas básicas...")
    positivos = ['positivo']
    negativos = ['negativo']
    neutros = ['neutral']

# 4. ¡YA ESTÁ LISTO!
print(f"\n🎉 Variables creadas: positivos, negativos, neutros")
print('positivos =',positivos)
print('negativos=',negativos)
print('neutros=',neutros)





C:/Users/marely/OneDrive/Documentos/Oracle_ONE/Hackaton/SentimentAPI-Project/sentiment-api/data-science/sources/diccionarios/sentimientos_mapeo.json
✅ JSON cargado: 109 sentimientos
📊 Positivos: 63, Negativos: 40, Neutros: 6

🎉 Variables creadas: positivos, negativos, neutros
positivos = ['aceptación', 'admiracion', 'adoración', 'agradecido', 'alegria', 'amabilidad', 'amistad', 'amor', 'apreciacion', 'armonia', 'asombro', 'cautivacion', 'celebracion', 'colorido', 'confiado', 'confianza', 'contentamiento', 'creatividad', 'cumplimiento', 'descubrimiento', 'deslumbrar', 'determinacion', 'disfrute', 'diversion', 'elacion', 'elegancia', 'emocion', 'empoderamiento', 'empatíco', 'encantamiento', 'energia', 'entusiasmo', 'esperanza', 'euforia', 'excitacion', 'felicidad', 'grandeza', 'gratitud', 'inspiracion', 'inspirado', 'intimidacion', 'jugueton', 'logro', 'maravilla', 'melodico', 'motivacion', 'optimismo', 'orgullo', 'positividad', 'positivo', 'reconfortante', 'resiliencia', 'resplandor', '

#### **Limpieza previa a la categorización**

In [236]:
# Aplicar la función genérica
positivos_limpios = limpiar_y_unificar_sentimientos(positivos,nombre_lista="sentimientos_positivos")

📊 Lista 'sentimientos_positivos' (sin limpiar): 63
Muestra (primeros 10): ['aceptación', 'admiracion', 'adoración', 'agradecido', 'alegria', 'amabilidad', 'amistad', 'amor', 'apreciacion', 'armonia']
📊 Lista 'sentimientos_positivos' (limpios): 63
Muestra (primeros 10): ['aceptación', 'admiracion', 'adoración', 'agradecido', 'alegria', 'amabilidad', 'amistad', 'amor', 'apreciacion', 'armonia']


In [237]:
# Aplicar la función genérica
negativos_limpios = limpiar_y_unificar_sentimientos(negativos,nombre_lista="sentimientos_negativos")

📊 Lista 'sentimientos_negativos' (sin limpiar): 40
Muestra (primeros 10): ['abrumado', 'aburrimiento', 'aislamiento', 'amargura', 'angustia', 'anhelo', 'ansiedad', 'aprensivo', 'arrepentimiento', 'asco']
📊 Lista 'sentimientos_negativos' (limpios): 40
Muestra (primeros 10): ['abrumado', 'aburrimiento', 'aislamiento', 'amargura', 'angustia', 'anhelo', 'ansiedad', 'aprensivo', 'arrepentimiento', 'asco']


In [238]:
# Aplicar la función genérica
neutros_limpios = limpiar_y_unificar_sentimientos(neutros,nombre_lista="sentimientos_neutros")

📊 Lista 'sentimientos_neutros' (sin limpiar): 6
Muestra (primeros 10): ['ambivalencia', 'anticipacion', 'curiosidad', 'irrelevante', 'neutral', 'sorpresa']
📊 Lista 'sentimientos_neutros' (limpios): 6
Muestra (primeros 10): ['ambivalencia', 'anticipacion', 'curiosidad', 'irrelevante', 'neutral', 'sorpresa']


In [239]:

categorias = [positivos_limpios, negativos_limpios, neutros_limpios]

# Verificación del total
total_clasificados = len(positivos_limpios) + len(negativos_limpios) + len(neutros_limpios)
print(f'\n✅ Total clasificado: {total_clasificados}/{len(sentimientos_unicos_limpios)}')
print(f'   - Positivos: {len(positivos_limpios)} ({len(positivos_limpios)/106*100:.1f}%)')
print(f'   - Negativos: {len(negativos_limpios)} ({len(negativos_limpios)/106*100:.1f}%)')
print(f'   - Neutros: {len(neutros_limpios)} ({len(neutros_limpios)/106*100:.1f}%)')
print(f'Total: {len(positivos_limpios) + len(negativos_limpios) + len(neutros_limpios)}')
print()

# Verificar si existen elementos en las listas que no se encuentran en la lista sentimientos_unicos
for sentimiento in negativos_limpios + positivos_limpios + neutros_limpios:
    if sentimiento not in sentimientos_unicos_limpios:
        print(f"❌ Sentimiento no encontrado en el dataset: {sentimiento}")


for sentimiento in sentimientos_unicos_limpios:
    if sentimiento not in positivos_limpios + negativos_limpios + neutros_limpios:
        print(f"❌ Sentimiento no clasificado: {sentimiento}")
# else:
#     print("✅ Todos los sentimientos del dataset están clasificados.")



✅ Total clasificado: 109/105
   - Positivos: 63 (59.4%)
   - Negativos: 40 (37.7%)
   - Neutros: 6 (5.7%)
Total: 109

❌ Sentimiento no encontrado en el dataset: decepcion
❌ Sentimiento no encontrado en el dataset: desesperacion
❌ Sentimiento no encontrado en el dataset: frustracion
❌ Sentimiento no encontrado en el dataset: lastima
❌ Sentimiento no encontrado en el dataset: melancolia
❌ Sentimiento no encontrado en el dataset: negativa
❌ Sentimiento no encontrado en el dataset: obstaculo
❌ Sentimiento no encontrado en el dataset: perdida
❌ Sentimiento no encontrado en el dataset: reflexion
❌ Sentimiento no encontrado en el dataset: traicion
❌ Sentimiento no encontrado en el dataset: admiracion
❌ Sentimiento no encontrado en el dataset: alegria
❌ Sentimiento no encontrado en el dataset: apreciacion
❌ Sentimiento no encontrado en el dataset: armonia
❌ Sentimiento no encontrado en el dataset: cautivacion
❌ Sentimiento no encontrado en el dataset: celebracion
❌ Sentimiento no encontrado e

#### **Función para categorizar sentimientos**

In [240]:
def categorizar_sentimiento(sentimiento, categorias):
    """
    Categoriza sentimientos solo si están en las listas definidas.
    Devuelve None para sentimientos no clasificados.
    """
    sent = str(sentimiento).strip().lower()

    if sent in positivos:
        return 'positivo'
    elif sent in negativos:
        return 'negativo'
    elif sent in neutros:
        return 'neutral'
    else:
        # Devolvemos None para posterior filtrado
        return None
    


In [241]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio
306,"Ojos envidiosos fijos en el premio dorado, una...",Envidioso,"Ojos envidiosos fijos en el premio dorado, una..."
23,Acabo de terminar una rutina de ejercicios des...,Positivo,Acabo de terminar una rutina de ejercicios des...
169,"Llega el aburrimiento, el día se siente infini...",Aburrimiento,"Llega el aburrimiento, el dia se siente infini..."


In [242]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio
2318,"Bueno, en unos añitos como que se les va a cum...",positivo,"Bueno, en unos añitos como que se les va a cum..."
1774,el ojo blindado que me has regalado me mira mal,positivo,el ojo blindado que me has regalado me mira mal
937,Es distinto estar solo a estar en solitario ??,negativo,Es distinto estar solo a estar en solitario ??


In [243]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio
26251,Ropa de la Hermandad,Positivo,Ropa de la Hermandad
21901,#VALUE!,Positivo,#VALUE!
42437,Informe de pubg fallido en.be/Qz00Pth41sw vía 2,Negativo,Informe de pubg fallido en.be/Qz00Pth41sw via 2


#### **Categorizar sentimientos**

In [244]:
df1_clean['Sentimiento_Final'] = df1_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df1_categorized = df1_clean[df1_clean['Sentimiento_Final'].notna()].copy()

df2_clean['Sentimiento_Final'] = df2_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df2_categorized = df2_clean[df2_clean['Sentimiento_Final'].notna()].copy()

df3_clean['Sentimiento_Final'] = df3_clean['sentimiento'].apply(
    lambda x: categorizar_sentimiento(x,categorias)
)

df3_categorized = df3_clean[df3_clean['Sentimiento_Final'].notna()].copy()

print(f"✅ df1: {len(df1_categorized)} registros categorizados")
print(f"✅ df2: {len(df2_categorized)} registros categorizados")
print(f"✅ df3: {len(df3_categorized)} registros categorizados")

✅ df1: 563 registros categorizados
✅ df2: 2540 registros categorizados
✅ df3: 74682 registros categorizados


In [245]:
df1_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
227,Al navegar por el laberinto de los pensamiento...,Neutral,Al navegar por el laberinto de los pensamiento...,neutral
371,Intrigado por la sinfonía de colores en una ex...,Neutral,Intrigado por la sinfonia de colores en una ex...,neutral
687,Recibió una calificación no tan buena en un pr...,Malo,Recibio una calificacion no tan buena en un pr...,negativo


In [246]:
df2_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
2291,"???????? ""Al mundo entero quiero dar un mensaj...",positivo,"???????? ""Al mundo entero quiero dar un mensaj...",positivo
480,"Tal vez te vea mañana, y eso me asusta demasia...",negativo,"Tal vez te vea mañana, y eso me asusta demasia...",negativo
361,Vamos a jugar a 20 corazones...bueno 20 son mu...,negativo,Vamos a jugar a 20 corazones...bueno 20 son mu...,negativo


In [247]:
df3_clean.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
6734,"¡Ay, todas esas polillas locas! Se agotaron en...",Negativo,"¡Ay, todas esas polillas locas! Se agotaron en...",negativo
58519,¡La diversión continúa! ¡Gracias por animarnos...,Irrelevante,¡La diversion continua! ¡Gracias por animarnos...,neutral
54008,El nuevo parche no puede durar más de 3 rondas...,Negativo,El nuevo parche no puede durar mas de 3 rondas...,negativo


### <font color=lightgreen size=12>Limpiar dataset unificado</font>

#### **Función limpieza dataset unificado**

In [248]:
def limpiar_dataset_unificado(data, verbose=True):
    """
    Limpia dataset unificado para análisis de sentimientos.

    Proceso:
    1. Identifica y elimina CONTRADICCIONES (textos con diferentes sentimientos)
    2. Elimina DUPLICADOS exactos (mismo texto, mismo sentimiento)
    3. Limpieza final (espacios vacíos, NaN)

    Args:
        data: DataFrame con 'Texto_Limpio' y 'Sentimiento_Final'
        verbose: Si True, muestra análisis detallado

    Returns:
        DataFrame limpio, sin duplicados ni contradicciones
    """

    if verbose:
        print("🧹 LIMPIANDO DATASET UNIFICADO")
        print("-" * 50)
        print(f"Registros iniciales: {len(data):,}")
        print(f"Textos únicos iniciales: {data['Texto_Limpio'].nunique():,}")

    # Hacer copia para no modificar original
    df = data.copy()

    # ===== 1. ELIMINAR CONTRADICCIONES (PRIMERO) =====
    if verbose:
        print(f"\n1. 🔍 BUSCANDO CONTRADICCIONES...")

    # Textos con más de un sentimiento diferente
    conteo_sentimientos = df.groupby('Texto_Limpio')['Sentimiento_Final'].nunique()
    textos_con_contradiccion = conteo_sentimientos[conteo_sentimientos > 1].index.tolist()

    if textos_con_contradiccion:
        if verbose:
            print(f"   ⚠️  Encontradas: {len(textos_con_contradiccion):,} contradicciones")

            # Mostrar algunos ejemplos
            print(f"   • Ejemplos (primeros 2):")
            for texto in textos_con_contradiccion[:2]:
                sentimientos = df[df['Texto_Limpio'] == texto]['Sentimiento_Final'].unique()
                texto_corto = texto[:60] + "..." if len(texto) > 60 else texto
                print(f"     - '{texto_corto}'")
                print(f"       → Sentimientos: {', '.join(sentimientos)}")

        # Eliminar TODOS los registros de textos contradictorios
        df_sin_contradicciones = df[~df['Texto_Limpio'].isin(textos_con_contradiccion)].copy()

        if verbose:
            eliminados = len(df) - len(df_sin_contradicciones)
            print(f"   🗑️  Eliminados: {eliminados:,} registros por contradicciones")
    else:
        if verbose:
            print(f"   ✅ No hay contradicciones")
        df_sin_contradicciones = df.copy()

    # ===== 2. ELIMINAR DUPLICADOS EXACTOS =====
    if verbose:
        print(f"\n2. 🔍 BUSCANDO DUPLICADOS EXACTOS...")

    # Contar duplicados exactos (mismo texto, mismo sentimiento)
    conteo_duplicados = df_sin_contradicciones['Texto_Limpio'].value_counts()
    textos_duplicados = conteo_duplicados[conteo_duplicados > 1].index.tolist()

    if textos_duplicados:
        if verbose:
            print(f"   ⚠️  Encontrados: {len(textos_duplicados):,} textos duplicados")

            # Calcular cuántos registros se eliminarán
            total_a_eliminar = sum([conteo_duplicados[t] - 1 for t in textos_duplicados])
            print(f"   • Registros a eliminar: {total_a_eliminar:,}")

        # Eliminar duplicados (mantener primera aparición)
        df_sin_duplicados = df_sin_contradicciones.drop_duplicates(
            subset=['Texto_Limpio'],
            keep='first'
        )

        if verbose:
            eliminados = len(df_sin_contradicciones) - len(df_sin_duplicados)
            print(f"   🗑️  Eliminados: {eliminados:,} registros duplicados")
    else:
        if verbose:
            print(f"   ✅ No hay duplicados exactos")
        df_sin_duplicados = df_sin_contradicciones.copy()

    # ===== 3. LIMPIEZA FINAL =====
    if verbose:
        print(f"\n3. 🧹 LIMPIEZA FINAL...")

    df_final = df_sin_duplicados.copy()

    # Filtrar solo columnas necesarias
    df_final = df_final[['Texto_Limpio', 'Sentimiento_Final']]

    # Eliminar textos vacíos o solo espacios
    textos_vacios_antes = len(df_final)
    df_final = df_final[df_final['Texto_Limpio'].str.strip() != ""]
    textos_vacios_eliminados = textos_vacios_antes - len(df_final)

    if verbose and textos_vacios_eliminados > 0:
        print(f"   • Textos vacíos eliminados: {textos_vacios_eliminados}")

    # Eliminar sentimientos NaN
    sentimientos_nan_antes = len(df_final)
    df_final = df_final[df_final['Sentimiento_Final'].notna()]
    sentimientos_nan_eliminados = sentimientos_nan_antes - len(df_final)

    if verbose and sentimientos_nan_eliminados > 0:
        print(f"   • Sentimientos NaN eliminados: {sentimientos_nan_eliminados}")

    # ===== 4. VERIFICACIÓN Y RESUMEN =====
    if verbose:
        print(f"\n4. ✅ VERIFICACIÓN FINAL")
        print(f"   • Registros finales: {len(df_final):,}")
        print(f"   • Textos únicos finales: {df_final['Texto_Limpio'].nunique():,}")

        # Verificar que cada texto aparece solo una vez
        if len(df_final) == df_final['Texto_Limpio'].nunique():
            print(f"   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez")
        else:
            diferencia = len(df_final) - df_final['Texto_Limpio'].nunique()
            print(f"   ⚠️  ¡Problema! Hay {diferencia} duplicados")

        # Resumen
        print(f"\n" + "=" * 50)
        print("📊 RESUMEN DE LIMPIEZA")
        print("=" * 50)

        total_eliminados = (len(data) - len(df_final))
        porcentaje_eliminado = (total_eliminados / len(data)) * 100

        print(f"Registros iniciales: {len(data):,}")
        print(f"Registros finales: {len(df_final):,}")
        print(f"Total eliminados: {total_eliminados:,} ({porcentaje_eliminado:.1f}%)")

        # Distribución de sentimientos
        print(f"\n📈 DISTRIBUCIÓN FINAL DE SENTIMIENTOS:")
        distribucion = df_final['Sentimiento_Final'].value_counts()
        for sentimiento, count in distribucion.items():
            porcentaje = (count / len(df_final)) * 100
            print(f"   • {sentimiento}: {count:,} ({porcentaje:.1f}%)")

    return df_final


In [249]:
df1_categorized.sample(3)
df2_categorized.sample(3)
df3_categorized.sample(3)

,texto,sentimiento,Texto_Limpio,Sentimiento_Final
53189,RED DEAD REDEMPTION 7..,Neutral,RED DEAD REDEMPTION 7..,neutral
5817,@amazon ¡Acabo de presionar sin querer la opci...,Neutral,@amazon ¡Acabo de presionar sin querer la opci...,neutral
25817,Â¡JODER CÃ“MO!,Positivo,A¡JODER CA“MO!,positivo


#### **Unificar datataset y limpieza**

In [250]:
print("=" * 70)
print("🔗 UNIFICANDO DATASETS CATEGORIZADOS")
print("=" * 70)

# Unificar los datasets categorizados
df_unificado = pd.concat([df1_categorized[['Texto_Limpio', 'Sentimiento_Final']], df2_categorized[['Texto_Limpio', 'Sentimiento_Final']], df3_categorized[['Texto_Limpio','Sentimiento_Final']]], ignore_index=True)

print(f"📦 Dataset unificado: {df_unificado.shape}")
print(f"   • Registros: {len(df_unificado):,}")
print(f"   • Textos únicos: {df_unificado['Texto_Limpio'].nunique():,}")


# %%
print("\n" + "=" * 70)
print("🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO")
print("=" * 70)

# Aplicar limpieza
df_final = limpiar_dataset_unificado(df_unificado, verbose=True)

🔗 UNIFICANDO DATASETS CATEGORIZADOS
📦 Dataset unificado: (77785, 2)
   • Registros: 77,785
   • Textos únicos: 69,306

🧹 APLICANDO LIMPIEZA AL DATASET UNIFICADO
🧹 LIMPIANDO DATASET UNIFICADO
--------------------------------------------------
Registros iniciales: 77,785
Textos únicos iniciales: 69,306

1. 🔍 BUSCANDO CONTRADICCIONES...
   ⚠️  Encontradas: 224 contradicciones
   • Ejemplos (primeros 2):
     - ''
       → Sentimientos: neutral, positivo, negativo
     - '!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!...'
       → Sentimientos: negativo, neutral
   🗑️  Eliminados: 3,141 registros por contradicciones

2. 🔍 BUSCANDO DUPLICADOS EXACTOS...
   ⚠️  Encontrados: 3,987 textos duplicados
   • Registros a eliminar: 5,562
   🗑️  Eliminados: 5,562 registros duplicados

3. 🧹 LIMPIEZA FINAL...

4. ✅ VERIFICACIÓN FINAL
   • Registros finales: 69,082
   • Textos únicos finales: 69,082
   🎯 ¡Dataset 100% limpio! Cada texto aparece solo una vez

📊 RESUMEN DE LIMPIEZA
Registros

In [251]:
df_unificado.sample(3)

,Texto_Limpio,Sentimiento_Final
62398,Los fanA¡ticos de una pelA cula conspirativa p...,neutral
7068,Si van al centro del satelite por cualquier mo...,neutral
18964,Tome el servidor del mar,positivo


 ### <font size=12 color=lightgreen>Análisis de Distribución y Visualización</font>

#### **Análisis de distribución de sentimientos**

In [252]:
#📊 ANÁLISIS DE DISTRIBUCIÓN DEL DATASET FINAL

print("=" * 60)
print("📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL")
print("=" * 60)

# 1. Calcular conteos y porcentajes
conteos = df_final['Sentimiento_Final'].value_counts()
total_registros = len(df_final)
porcentajes = (conteos / total_registros * 100).round(2)

# 2. Mostrar tabla detallada
print(f"{'SENTIMIENTO':<12} | {'CANTIDAD':>8} | {'PORCENTAJE':>10} | {'PROPORCIÓN'}")
print("-" * 50)

for sentimiento in ['positivo', 'negativo', 'neutral']:
    if sentimiento in conteos:
        count = conteos[sentimiento]
        porcentaje = porcentajes[sentimiento]
        # Crear barra visual
        barra = '█' * int(count / total_registros * 40)  # Escala a 40 caracteres
        print(f"{sentimiento.capitalize():<12} | {count:>8} | {porcentaje:>9}% | {barra}")

print("-" * 50)
print(f"{'TOTAL':<12} | {total_registros:>8} | {'100.00':>9}% | {'█' * 40}")
print("-" * 58)


📈 ANÁLISIS DE DISTRIBUCIÓN - DATASET FINAL
SENTIMIENTO  | CANTIDAD | PORCENTAJE | PROPORCIÓN
--------------------------------------------------
Positivo     |    19080 |     27.62% | ███████████
Negativo     |    21287 |     30.81% | ████████████
Neutral      |    28715 |     41.57% | ████████████████
--------------------------------------------------
TOTAL        |    69082 |    100.00% | ████████████████████████████████████████
----------------------------------------------------------


#### **Visualización de la distribución de Sentimientos**

In [253]:
# Grafica de pastel con Plotly

valores = df_final['Sentimiento_Final'].value_counts().reset_index()
valores.columns = ['sentimientos', 'Cantidad']
fig1 = px.pie(
    names = valores.sentimientos,
    values = valores.Cantidad,
)

fig1.update_traces(textposition='inside', textinfo='label+percent',  insidetextfont=dict(color = 'white', size=14)
)

fig1.update_layout(
    title_text=f'<b>Distribución de Sentimientos</b><br><span style="font-size:14px">Dataset Final: {total_registros} registros</span>',
    title_x=0.5,
    width=500,
    height=500,
    showlegend=False,
)

fig1.show()

### <font size=12 color=lightgreen> Exportar dataset </font>

#### **Definir ruta de exportación**

In [254]:
# Ruta actual
ruta_actual = Path.cwd()

# Buscar data-science
if ruta_actual.name == 'notebooks':
    # Si estamos en notebooks/, ir a ../datasets
    carpeta_datasets = ruta_actual.parent / 'datasets'
else:
    # Buscar data-science en directorios padres
    for directorio_padre in ruta_actual.parents:
        if (directorio_padre / 'data-science').exists():
            carpeta_datasets = directorio_padre / 'data-science' / 'datasets'
            break
    else:
        # Si no encuentra, usar directorio actual/datasets
        carpeta_datasets = ruta_actual / 'datasets'

# Crear carpeta si no existe
carpeta_datasets.mkdir(parents=True, exist_ok=True)

# Ruta completa del archivo
archivo_final = carpeta_datasets / 'dataset.csv'


#### **Exportar dataset**

In [255]:
# Renombrar columnas para formato final
df_exportar = df_final.rename({
    'Texto_Limpio': 'texto',
    'Sentimiento_Final': 'sentimiento'
}, axis=1)


metadata = {
    "total_registros": len(df_exportar),
    "distribucion": dict(df_exportar['sentimiento'].value_counts()),
    "fecha_creacion": datetime.now().isoformat(),
    "version": "2.0.0",
    "fuentes": [
        "sentimentdataset_es.csv",
        "sentiment_analysis_dataset.csv",
        "twitter_trainnig.csv"    ]
}

# Exportar
df_exportar.to_csv(archivo_final, index=False, encoding='utf-8-sig')
print(f"✅ Dataset exportado: {archivo_final}")
print(f"📊 Registros: {len(df_exportar):,}")

# Crear copia para trabajo posterior
df = df_exportar.copy()

✅ Dataset exportado: c:\Users\marely\OneDrive\Documentos\Oracle_ONE\Hackaton\SentimentAPI-Project\sentiment-api\data-science\datasets\dataset.csv
📊 Registros: 69,082


#### **Verificar exportación**

In [256]:
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    Y verificación de integridad mejorada
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            # Probar con 5 filas primero
            df_test = pd.read_csv(ruta, encoding=enc, nrows=5)

            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                # 🔍 VERIFICACIÓN DE INTEGRIDAD MEJORADA
                print("🔍 Verificación de integridad:")
                print(f"   • Valores nulos totales: {df.isnull().sum().sum()}")
                print(f"   • Textos vacíos: {(df['texto'].str.strip() == '').sum()}")

                # Verificar que todos los sentimientos sean válidos
                sentimientos_validos = ['positivo', 'negativo', 'neutral']
                sentimientos_invalidos = df[~df['sentimiento'].isin(sentimientos_validos)]

                if len(sentimientos_invalidos) > 0:
                    print(f"   ⚠️  Sentimientos inválidos: {len(sentimientos_invalidos)}")
                    print(f"      Valores únicos inválidos: {sentimientos_invalidos['sentimiento'].unique()}")
                else:
                    print(f"   ✅ Todos los sentimientos son válidos")

                # Verificar unicidad
                textos_unicos = df['texto'].nunique()
                if len(df) == textos_unicos:
                    print(f"   ✅ 100% textos únicos: {textos_unicos:,} textos únicos")
                else:
                    print(f"   ⚠️  Duplicados: {len(df) - textos_unicos:,} textos duplicados")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df

            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

In [257]:
# Uso simple - así debería funcionar
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 69,082 registros (encoding: utf-8-sig)
🔍 Verificación de integridad:
   • Valores nulos totales: 0
   • Textos vacíos: 0
   ✅ Todos los sentimientos son válidos
   ✅ 100% textos únicos: 69,082 textos únicos
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                       texto sentimiento
¡Disfrutando de un hermoso dia en el parque!    positivo
        Esta mañana el trafico era terrible.    negativo


In [258]:
# Verificar que el archivo se pueda leer
def verificar_csv_simple(ruta_archivo, mostrar_muestra=True):
    """
    Verificación simplificada con detección de encoding
    """
    ruta = Path(ruta_archivo)

    if not ruta.exists():
        print(f"❌ Archivo no encontrado: {ruta}")
        return None

    # Detectar encoding
    encodings = ['utf-8-sig', 'utf-8', 'latin1', 'cp1252']

    for enc in encodings:
        try:
            df = pd.read_csv(ruta, encoding=enc, nrows=5)  # Probar con 5 filas
            # Si llegamos aquí, el encoding funciona
            try:
                # Ahora cargar completo
                df = pd.read_csv(ruta, encoding=enc)
                print(f"✅ CSV cargado: {len(df):,} registros (encoding: {enc})")

                if mostrar_muestra:
                    print(f"📝 Columnas: {list(df.columns)}")
                    print(f"📊 Muestra (2 filas):")
                    print(df.head(2).to_string(index=False))

                return df
            except Exception as e:
                print(f"❌ Error cargando con encoding {enc}: {type(e).__name__}")
                continue

        except UnicodeDecodeError:
            continue

    print("❌ No se pudo cargar con ningún encoding común")
    return None

# Uso simple
df_check = verificar_csv_simple(archivo_final, mostrar_muestra=True)

✅ CSV cargado: 69,082 registros (encoding: utf-8-sig)
📝 Columnas: ['texto', 'sentimiento']
📊 Muestra (2 filas):
                                       texto sentimiento
¡Disfrutando de un hermoso dia en el parque!    positivo
        Esta mañana el trafico era terrible.    negativo


 ### <font size=12 color=lightgreen> Resumen ejecutivo </font>

In [259]:
print("=" * 70)
print("📋 RESUMEN EJECUTIVO - HACKATHON SENTIMENT API")
print("=" * 70)
print(f"✅ Dataset final: {len(df_exportar):,} registros")
print(f"✅ Distribución balanceada: {porcentajes['positivo']}% 👍 | {porcentajes['negativo']}% 👎 | {porcentajes['neutral']}% 😐")
print(f"✅ Calidad del dataset:")
print(f"   • 0 contradicciones (cada texto tiene único sentimiento)")
print(f"   • 0 duplicados (100% textos únicos)")
print(f"   • 0 valores nulos")

📋 RESUMEN EJECUTIVO - HACKATHON SENTIMENT API
✅ Dataset final: 69,082 registros
✅ Distribución balanceada: 27.62% 👍 | 30.81% 👎 | 41.57% 😐
✅ Calidad del dataset:
   • 0 contradicciones (cada texto tiene único sentimiento)
   • 0 duplicados (100% textos únicos)
   • 0 valores nulos


---
### <font size=12 color=lightgreen>Observaciones</font>


### 1. **<font color='lightgreen'>Origen de los datos</font>**

Con el objetivo de mejorar la capacidad de generalización del modelo, se trabajó con dos datasets independientes obtenidos desde Kaggle.
Si bien ambos conjuntos de datos abordan el análisis de sentimiento en español, presentan diferencias en estructura, calidad lingüística y formato de origen. Su integración permitió ampliar la diversidad de expresiones textuales, reduciendo el sesgo hacia un único estilo de redacción y fortaleciendo la robustez del pipeline de preparación de datos en escenarios similares a producción.

#### **Fuentes de datos (Kaggle):**

- https://www.kaggle.com/datasets/engineercolsoquas/spanish-sentiment-analysis-dataset

- https://www.kaggle.com/datasets/kashishparmar02/social-media-sentiments-analysis-dataset

- https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis


---
### 2. **<font color='lightgreen'> Informe de Desafíos Técnicos y Soluciones</font>**

#### **Dataset** 1 – Inconsistencias en el idioma

- Problema: El dataset original presentaba traducciones incompletas, combinando registros en español con fragmentos en su idioma original, además de traducciones literales de baja calidad. Esta situación afectaba la coherencia semántica del texto y podía introducir ruido en el análisis de sentimiento.

- Solución aplicada: Se utilizó la herramienta de Traducción de Microsoft Excel como apoyo para identificar registros no traducidos. No obstante, la corrección se realizó de forma manual y supervisada, revisando y ajustando cada registro individualmente con el fin de preservar el significado original del texto y evitar distorsiones semánticas. Posteriormente, se realizó una revisión manual (sanity check) para asegurar la consistencia lingüística del dataset completo.

- Impacto en el análisis: La normalización del idioma permitió obtener un corpus coherente en español, reduciendo ambigüedades y mejorando la calidad de los datos de entrada para la etapa de clasificación de sentimiento.


**Dataset 2 – Problemas de codificación de caracteres (encoding)**

- Problema:
El segundo dataset se encontraba en formato Excel y presentaba errores de codificación al ser abierto, evidenciados por la aparición de caracteres especiales incorrectos (mojibake), lo que impedía un procesamiento adecuado del texto.

- Solución aplicada:
Como primer paso, el archivo fue exportado a formato CSV. Posteriormente, se realizó la ingesta mediante Power Query, donde se configuró explícitamente la codificación Unicode (UTF-8), corrigiendo la estructura de caracteres antes de su integración al pipeline de preparación de datos.

- Impacto en el análisis:
La corrección del encoding aseguró la correcta interpretación de caracteres propios del idioma español, evitando pérdidas de información y mejorando la calidad del texto procesado.
---


### 3. **<font color='lightgreen'>Normalización y Limpieza de Texto</font>**
- Se aplicó una función de preprocesamiento (limpiar_texto_sentimiento) que incluyó:

- Preservación de mayúsculas/minúsculas (para mantener intensidad emocional).

- Eliminación de tildes (pero conservación de ñ/Ñ).

- Limpieza de URLs, menciones y caracteres no imprimibles.

- Normalización de espacios y saltos de línea.

**Nota: Se decidió no convertir todo a minúsculas para conservar pistas contextuales (ej. “¡GENIAL!” vs. “genial”), relevantes para modelos basados en intensidad emocional.**

---

In [260]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69082 entries, 0 to 77784
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   texto        69082 non-null  object
 1   sentimiento  69082 non-null  object
dtypes: object(2)
memory usage: 1.6+ MB


 ### <font size=12 color=lightgreen> Machine Learning</font>

In [261]:
df.sample()

,texto,sentimiento
14615,Eso fue absolutamente genial por parte de Xbox...,positivo


In [262]:
# Descargar stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords

def limpiar_texto_mejorado(texto):
    """
    Limpieza de texto más conservadora que preserva palabras  de negación
    y modificadores de intensidad.
    """
    texto = texto.lower()

    # Eliminar caracteres especiales PERO preservar puntuación emocional
    texto = re.sub(r'[^a-záéíóúñ0-9\s!?.,\-]', '', texto)

    # Stopwords españolas
    stop_words = set(stopwords.words('spanish'))

    # PALABRAS CRÍTICAS A MANTENER (expandida vs. original)
    palabras_criticas = {
        # Negaciones
        'no', 'ni', 'sin', 'nunca', 'jamas', 'tampoco', 'nada', 'nadie',
        # Intensificadores
        'muy', 'mucho', 'poco', 'mas', 'menos', 'demasiado', 'bastante',
        # Modales
        'pero', 'aunque', 'sino', 'si',
        # Adjetivos de sentimiento
        'malo', 'mala', 'mal', 'bien', 'bueno', 'buena', 'mejor', 'peor',
        'horrible', 'terrible', 'excelente', 'pesimo', 'pésimo',
        # Verbos de sentimiento
        'odio', 'amo', 'encanta', 'disgusta', 'molesta',
        # Otros
        'contra', 'hacia'
    }

    # Remover stopwords EXCEPTO las críticas
    stop_words = stop_words - palabras_criticas

    palabras = texto.split()
    palabras_filtradas = [palabra for palabra in palabras if palabra not in stop_words]

    return ' '.join(palabras_filtradas)

# Aplicar limpieza mejorada
df['texto'] = df['texto'].apply(limpiar_texto_mejorado)

print("✅ Limpieza de texto mejorada aplicada")
print(f"\nMuestra de textos limpios:")
print(df[['texto', 'sentimiento']].sample(5))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marely\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✅ Limpieza de texto mejorada aplicada

Muestra de textos limpios:
                                                   texto sentimiento
17781  gameflip re alguien abusa codigo descuento mer...     neutral
22496  primera vez vida, paparazzi atacaron despues c...     neutral
32894  clasificacian tra brandon rob. juegos accian a...     neutral
33971                          recien enfrentado ceguera    negativo
8536   informacian, algoritmos no estan dando cradito...    negativo


In [263]:
df.sample(5)

,texto,sentimiento
23672,arrastrandose...,neutral
61868,facebook lleva segundo dia impidiendome compar...,negativo
24400,personalmente creo comunidad csgo gran problem...,negativo
29343,no puedo esperar credo asesino valhalla.,positivo
11290,playoverwatch desconecto cada juego juegos alg...,negativo


 ### <font size=12 color=lightgreen> Balanceo del Dataset, TF-IDF, Modelo, Métricas y Serialización </font>

### Instalación de `imblearn`

Primero, necesitamos instalar la librería `imblearn`, que proporciona herramientas para manejar datasets desbalanceados, incluyendo la técnica SMOTE para sobremuestreo.

In [264]:
get_ipython().system('pip install imblearn') # type: ignore
print("Librería 'imblearn' instalada exitosamente.")

Librería 'imblearn' instalada exitosamente.


### Separación de Características y Target

Ahora, separaremos las características (el texto limpio) y la variable objetivo (el sentimiento) de nuestro DataFrame `df`. También mostraremos la distribución inicial de las clases para ver el desbalanceo.

In [265]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import joblib
import json
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [266]:
# Separar X e y
X = df['texto']
y = df['sentimiento']

print("Distribución de clases:")
print(y.value_counts())

# División train/test
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTrain: {len(X_train_unbalanced)} | Test: {len(X_test)}")

# 🆕 TF-IDF MEJORADO
vectorizer = TfidfVectorizer(
    max_features=10000,      # ⬆ Aumentado de 5000
    ngram_range=(1, 3),      # ⬆ Trigramas (antes solo bigramas)
    min_df=2,                # 🆕 Nuevo
    max_df=0.95,             # 🆕 Nuevo
    sublinear_tf=True        # 🆕 Nuevo
)

X_train_tfidf_unbalanced = vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = vectorizer.transform(X_test)

print(f"\n✅ Vectorización completada")
print(f"   Dimensiones: {X_train_tfidf_unbalanced.shape}")
print(f"   Vocabulario: {len(vectorizer.get_feature_names_out())} términos")

Distribución de clases:
sentimiento
neutral     28715
negativo    21287
positivo    19080
Name: count, dtype: int64

Train: 55265 | Test: 13817

✅ Vectorización completada
   Dimensiones: (55265, 10000)
   Vocabulario: 10000 términos


### <font size=12 color=lightgreen>División de Datos (Entrenamiento y Prueba) y Vectorización TF-IDF</font>



Es crucial dividir el dataset en conjuntos de entrenamiento y prueba *antes* de aplicar SMOTE para evitar la fuga de datos (data leakage). Luego, transformaremos los textos en vectores numéricos usando `TfidfVectorizer`.

In [267]:
# Dividir el dataset en conjuntos de entrenamiento y prueba ANTES de aplicar SMOTE
X_train_unbalanced, X_test, y_train_unbalanced, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nTamaño del conjunto de entrenamiento (desbalanceado): {len(X_train_unbalanced)} muestras")
print(f"Tamaño del conjunto de prueba: {len(X_test)} muestras")
print(f"Distribución de clases en el conjunto de entrenamiento (desbalanceado):\n{y_train_unbalanced.value_counts()}")
print(f"Distribución de clases en el conjunto de prueba:\n{y_test.value_counts()}")

# Inicializar TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000,ngram_range=(1,2)) # Limitando las características para eficiencia

# Ajustar y transformar X_train_unbalanced, y transformar X_test
X_train_tfidf_unbalanced = tfidf_vectorizer.fit_transform(X_train_unbalanced)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("\nVectorización TF-IDF completada en la división desbalanceada.")
print(f"Forma de X_train_tfidf_unbalanced: {X_train_tfidf_unbalanced.shape}")
print(f"Forma de X_test_tfidf: {X_test_tfidf.shape}")


Tamaño del conjunto de entrenamiento (desbalanceado): 55265 muestras
Tamaño del conjunto de prueba: 13817 muestras
Distribución de clases en el conjunto de entrenamiento (desbalanceado):
sentimiento
neutral     22972
negativo    17029
positivo    15264
Name: count, dtype: int64
Distribución de clases en el conjunto de prueba:
sentimiento
neutral     5743
negativo    4258
positivo    3816
Name: count, dtype: int64

Vectorización TF-IDF completada en la división desbalanceada.
Forma de X_train_tfidf_unbalanced: (55265, 5000)
Forma de X_test_tfidf: (13817, 5000)


### <font size=12 color=lightgreen> Balanceo del Conjunto de Entrenamiento con SMOTE</font>

Ahora aplicaremos SMOTE solo al conjunto de entrenamiento vectorizado (`X_train_tfidf_unbalanced`) para balancear las clases, generando muestras sintéticas para las clases minoritarias.

In [268]:
smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf_unbalanced, y_train_unbalanced)

print("✅ SMOTE aplicado")
print(f"\nDistribución después de SMOTE:")
print(pd.Series(y_train).value_counts())
print(f"\nNuevas dimensiones: {X_train_tfidf.shape}")

✅ SMOTE aplicado

Distribución después de SMOTE:
sentimiento
neutral     22972
negativo    22972
positivo    22972
Name: count, dtype: int64

Nuevas dimensiones: (68916, 5000)


### <font size=12 color=lightgreen> Entrenamiento del Modelo</font>


# Entrenamiento de Múltiples Modelos

Entrenaremos 3 modelos y compararemos:
1. **Logistic Regression** (baseline mejorado)
2. **SVM** (Support Vector Machine)
3. **Random Forest**

In [269]:
# # 1. Logistic Regression (mejorado)
# print("[1/3] Entrenando Logistic Regression...")
# model_lr = LogisticRegression(
#     max_iter=2000,
#     random_state=42,
#     C=1.0,
#     solver='lbfgs',
#     multi_class='multinomial'
# )
# model_lr.fit(X_train_tfidf, y_train)
# print("✅ Logistic Regression entrenado")

# # 2. SVM
# print("\n[2/3] Entrenando SVM...")
# model_svm = SVC(
#     kernel='linear',
#     C=1.0,
#     probability=True,
#     random_state=42,
#     class_weight='balanced'
# )
# model_svm.fit(X_train_tfidf, y_train)
# print("✅ SVM entrenado")

# # 3. Random Forest
# print("\n[3/3] Entrenando Random Forest...")
# model_rf = RandomForestClassifier(
#     n_estimators=200,
#     max_depth=None,
#     min_samples_split=5,
#     min_samples_leaf=2,
#     random_state=42,
#     n_jobs=-1,
#     class_weight='balanced'
# )
# model_rf.fit(X_train_tfidf, y_train)
# print("✅ Random Forest entrenado")

# print("\n" + "="*60)
# print("✅ TODOS LOS MODELOS ENTRENADOS")
# print("="*60)

### <font size=12 color=lightgreen>Evaluación de Modelos:</font>

Evaluaremos el rendimiento del modelo en el conjunto de prueba utilizando métricas clave como accuracy, precision, recall y F1-score.

In [270]:
def evaluar_modelo(model, X_test, y_test, nombre):
    """Evaluación detallada de un modelo"""
    print(f"\n{'='*60}")
    print(f"EVALUACIÓN: {nombre}")
    print(f"{'='*60}")

    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n📊 Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

    print(f"\n📋 Reporte de Clasificación:")
    print(classification_report(y_test, y_pred, digits=4))

    print(f"\n🔢 Matriz de Confusión:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    # Probabilidad promedio
    predicciones_correctas = y_pred == y_test
    probabilidades_correctas = []

    for i, correcto in enumerate(predicciones_correctas):
        if correcto:
            clase_predicha = y_pred[i]
            clase_idx = list(model.classes_).index(clase_predicha)
            probabilidades_correctas.append(y_pred_proba[i][clase_idx])

    if probabilidades_correctas:
        prob_promedio = np.mean(probabilidades_correctas)
        print(f"\n💯 Probabilidad promedio (correctas): {prob_promedio:.4f} ({prob_promedio*100:.2f}%)")

    return accuracy

# Evaluar todos los modelos
acc_lr = evaluar_modelo(model_lr, X_test_tfidf, y_test, "Logistic Regression (Mejorado)")
acc_svm = evaluar_modelo(model_svm, X_test_tfidf, y_test, "SVM")
acc_rf = evaluar_modelo(model_rf, X_test_tfidf, y_test, "Random Forest")

NameError: name 'model_lr' is not defined

### <font size=12 color=lightgreen> Serialización del Modelo y Vectorizadors</font>



Guardaremos el modelo entrenado y el objeto `TfidfVectorizer` utilizando `joblib` para poder reutilizarlos más tarde en la API de predicción.

In [ ]:
# Based on evaluation, SVM was the best model, so we'll use it for serialization.
mejor_modelo = model_svm

# Serializar el Modelo y el Vectorizador
joblib.dump(mejor_modelo, '/content/modelo_sentimientos.pkl')
joblib.dump(tfidf_vectorizer, '/content/vectorizador.pkl')

print("\nModelo y vectorizador guardados exitosamente en '/content/modelo_sentimientos.pkl' y '/content/vectorizador.pkl'.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/modelo_sentimientos.pkl'

In [ ]:
# Tabla comparativa
comparacion = pd.DataFrame({
    'Modelo': ['Logistic Regression', 'SVM', 'Random Forest'],
    'Accuracy': [acc_lr, acc_svm, acc_rf]
})

comparacion['Accuracy %'] = comparacion['Accuracy'].apply(lambda x: f"{x*100:.2f}%")
comparacion['Mejora vs. Original'] = comparacion['Accuracy'].apply(lambda x: f"+{(x - 0.79)*100:.2f}%")
comparacion = comparacion.sort_values('Accuracy', ascending=False)

print("\n" + "="*70)
print("📊 COMPARACIÓN DE MODELOS")
print("="*70)
print(comparacion.to_string(index=False))
print("\n🔸 Modelo original (baseline): 79.00%")
print("="*70)

mejor_accuracy = comparacion['Accuracy'].max()
if mejor_accuracy >= 0.83:
    print("\n✅ META DE FASE 1 ALCANZADA (83-85%)")
else:
    print(f"\n⚠️ Casi alcanzado (falta {(0.83 - mejor_accuracy)*100:.2f}%)")

comparacion

### <font size=12 color=lightgreen>Pruebas con Casos Específicos</font>

Validar que ahora clasifica correctamente los casos problemáticos

In [ ]:
# Seleccionar el mejor modelo
modelos_dict = {
    'Logistic Regression': model_lr,
    'SVM': model_svm,
    'Random Forest': model_rf
}

nombre_mejor = comparacion.iloc[0]['Modelo']
mejor_modelo = modelos_dict[nombre_mejor]

print(f"🏆 Mejor modelo: {nombre_mejor}")
print(f"📊 Accuracy: {comparacion.iloc[0]['Accuracy %']}")

# Casos de prueba
casos_prueba = [
    ("mala atención", "negativo"),
    ("mal comportamiento de los empleado", "negativo"),
    ("la empresa esta perdida en lo que hace", "negativo"),
    ("pésimo servicio", "negativo"),
    ("nunca vuelvo", "negativo"),
    ("excelente servicio", "positivo"),
    ("me encantó", "positivo"),
    ("muy buena atención", "positivo"),
    ("es normal, nada especial", "neutral"),
    ("está bien", "neutral"),
]

print("\n" + "="*70)
print("PRUEBAS CON CASOS ESPECÍFICOS")
print("="*70)

aciertos = 0

for texto, esperado in casos_prueba:
    # Preprocesar
    texto_limpio = limpiar_texto_mejorado(texto)

    # Vectorizar y predecir
    texto_vectorizado = tfidf_vectorizer.transform([texto_limpio])
    prediccion = mejor_modelo.predict(texto_vectorizado)[0]
    probabilidades = mejor_modelo.predict_proba(texto_vectorizado)[0]

    clase_idx = list(mejor_modelo.classes_).index(prediccion)
    prob_prediccion = probabilidades[clase_idx]

    es_correcto = prediccion == esperado
    if es_correcto:
        aciertos += 1
        emoji = "✅"
    else:
        emoji = "❌"

    print(f"\n{emoji} '{texto}'")
    print(f"   Esperado: {esperado} | Predicho: {prediccion} | Confianza: {prob_prediccion*100:.2f}%")

print("\n" + "="*70)
print(f"RESULTADO: {aciertos}/{len(casos_prueba)} correctos ({aciertos/len(casos_prueba)*100:.1f}%")
print("="*70)

### <font size=12 color=lightgreen>Exportación del modelo</font>

In [ ]:
from pyexpat import model
from sklearn.pipeline import Pipeline
import joblib

# Creamos un Pipeline manual uniendo las dos piezas
pipeline_para_produccion = Pipeline([
    ('vectorizer', tfidf_vectorizer), # Primero transforma el texto a números
    ('classifier', model)             # Luego predice con esos números
])

# Probamos que funcione antes de exportar
test_text = ["Este es un ejemplo de prueba para ver si funciona el pipeline"]
prediccion = pipeline_para_produccion.predict(test_text)
print(f"Prueba del pipeline: {prediccion}")

# EXPORTAR EL ARCHIVO FINAL
# Este es el archivo que debes subir a la carpeta de tu microservicio
joblib.dump(pipeline_para_produccion, 'modelo_entrenado.joblib')

print("✅ Archivo 'modelo_entrenado.joblib' creado exitosamente.")

********************************************************************************************************************************************************************************************************************